In [1]:
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Subset
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.amp import GradScaler, autocast
import os
import random
from torch.utils.data import DataLoader, SubsetRandomSampler

In [2]:
import torch
import numpy as np
from torch.utils.data import DataLoader, Subset
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import random

# Define dataset root directory
mnist_root = '/home/j597s263/scratch/j597s263/Datasets/MNIST'

random.seed(42)
torch.manual_seed(42)
np.random.seed(42)

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor()
])

train_dataset = datasets.MNIST(root=mnist_root, transform=transform, train=True, download=True)
test_dataset = datasets.MNIST(root=mnist_root, transform=transform, train=False, download=True)

train_indices = list(range(len(train_dataset)))
random.shuffle(train_indices)  

split_idx = int(0.9 * len(train_indices))  
train_indices, attack_indices = train_indices[:split_idx], train_indices[split_idx:]

train_data = Subset(train_dataset, train_indices)
attack_data = Subset(train_dataset, attack_indices)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)  # Shuffle within batches
attack_loader = DataLoader(attack_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Total training samples: {len(train_dataset)}")
print(f"Training samples after split: {len(train_data)}")
print(f"Attack samples: {len(attack_data)}")
print(f"Testing samples: {len(test_dataset)}")

Total training samples: 60000
Training samples after split: 54000
Attack samples: 6000
Testing samples: 10000


In [3]:
import torch
import torch.nn as  nn
import torch.nn.functional as F


class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Bottleneck, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)
        
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
        self.batch_norm3 = nn.BatchNorm2d(out_channels*self.expansion)
        
        self.i_downsample = i_downsample
        self.stride = stride
        self.relu = nn.ReLU()
        
    def forward(self, x):
        identity = x.clone()
        x = self.relu(self.batch_norm1(self.conv1(x)))
        
        x = self.relu(self.batch_norm2(self.conv2(x)))
        
        x = self.conv3(x)
        x = self.batch_norm3(x)
        
        #downsample if needed
        if self.i_downsample is not None:
            identity = self.i_downsample(identity)
        #add identity
        x+=identity
        x=self.relu(x)
        
        return x

class Block(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Block, self).__init__()
       

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, stride=stride, bias=False)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)

        self.i_downsample = i_downsample
        self.stride = stride
        self.relu = nn.ReLU()

    def forward(self, x):
      identity = x.clone()

      x = self.relu(self.batch_norm2(self.conv1(x)))
      x = self.batch_norm2(self.conv2(x))

      if self.i_downsample is not None:
          identity = self.i_downsample(identity)
      print(x.shape)
      print(identity.shape)
      x += identity
      x = self.relu(x)
      return x


        
        
class ResNet(nn.Module):
    def __init__(self, ResBlock, layer_list, num_classes, num_channels=3):
        super(ResNet, self).__init__()
        self.in_channels = 64
        
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size = 3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(ResBlock, layer_list[0], planes=64)
        self.layer2 = self._make_layer(ResBlock, layer_list[1], planes=128, stride=2)
        self.layer3 = self._make_layer(ResBlock, layer_list[2], planes=256, stride=2)
        self.layer4 = self._make_layer(ResBlock, layer_list[3], planes=512, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512*ResBlock.expansion, num_classes)
        
    def forward(self, x):
        x = self.relu(self.batch_norm1(self.conv1(x)))
        x = self.max_pool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        
        return x
        
    def _make_layer(self, ResBlock, blocks, planes, stride=1):
        ii_downsample = None
        layers = []
        
        if stride != 1 or self.in_channels != planes*ResBlock.expansion:
            ii_downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, planes*ResBlock.expansion, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes*ResBlock.expansion)
            )
            
        layers.append(ResBlock(self.in_channels, planes, i_downsample=ii_downsample, stride=stride))
        self.in_channels = planes*ResBlock.expansion
        
        for i in range(blocks-1):
            layers.append(ResBlock(self.in_channels, planes))
            
        return nn.Sequential(*layers)

        
        
def ResNet50(num_classes, channels=1):
    return ResNet(Bottleneck, [3,4,6,3], num_classes, channels)

In [4]:
# Load the model
import torch

# Define the path to the model
device = "cuda" 

# Load the model
model = torch.load('/home/j597s263/scratch/j597s263/Models/Resnet/Base/ResMNIBase.mod', weights_only=False, map_location="cuda")
model = model.to(device)
model.eval()  

print("Model loaded successfully!")

Model loaded successfully!


In [5]:
import torch
import numpy as np
from captum.attr import IntegratedGradients
import os

# Ensure the model is in evaluation mode and on the correct device
device = 'cuda'
model.to(device)
model.eval()

# Define the Integrated Gradients method
ig = IntegratedGradients(model)

# Function to compute and save explanations as .npy
def compute_and_save_explanations_with_labels(data_loader, save_dir):
    """
    Compute Integrated Gradient explanations for all images in the dataset and save as .npy files.

    Args:
        data_loader (DataLoader): DataLoader for the dataset to explain.
        save_dir (str): Directory to save the explanations as .npy files.
    """
    os.makedirs(save_dir, exist_ok=True)  # Ensure the directory exists

    for batch_idx, (images, labels) in enumerate(data_loader):
        images, labels = images.to(device), labels.to(device)

        # Compute attributions using IG
        attributions, delta = ig.attribute(
            inputs=images,
            target=labels,  # Use ground-truth labels for explanations
            baselines=torch.zeros_like(images).to(device),  # Baseline: black image
            return_convergence_delta=True,
            n_steps = 10
        )

        # Save explanations for each image in the batch
        for i in range(images.size(0)):
            label = labels[i].item()  # Get the label as an integer
            explanation = attributions[i].cpu().numpy()  # Shape: (1, 224, 224)

            # Create a label array of shape (1, 224, 224)
            label_array = np.full((1, explanation.shape[1], explanation.shape[2]), label, dtype=np.float32)

            # Concatenate label and explanation to shape (2, 224, 224) (for grayscale images)
            explanation_with_label = np.concatenate([label_array, explanation], axis=0)

            # Save as .npy
            save_path = os.path.join(save_dir, f"explanation_{batch_idx * images.size(0) + i}.npy")
            np.save(save_path, explanation_with_label)

        print(f"Processed batch {batch_idx + 1}/{len(data_loader)}")

    print(f"Explanations saved to {save_dir}")

# Define the directory to save explanations
save_dir = "/home/j597s263/scratch/j597s263/Datasets/Explanation_values/Resnet/IG_ResMNI"

# Compute and save explanations
compute_and_save_explanations_with_labels(attack_loader, save_dir)

Processed batch 1/188
Processed batch 2/188
Processed batch 3/188
Processed batch 4/188
Processed batch 5/188
Processed batch 6/188
Processed batch 7/188
Processed batch 8/188
Processed batch 9/188
Processed batch 10/188
Processed batch 11/188
Processed batch 12/188
Processed batch 13/188
Processed batch 14/188
Processed batch 15/188
Processed batch 16/188
Processed batch 17/188
Processed batch 18/188
Processed batch 19/188
Processed batch 20/188
Processed batch 21/188
Processed batch 22/188
Processed batch 23/188
Processed batch 24/188
Processed batch 25/188
Processed batch 26/188
Processed batch 27/188
Processed batch 28/188
Processed batch 29/188
Processed batch 30/188
Processed batch 31/188
Processed batch 32/188
Processed batch 33/188
Processed batch 34/188
Processed batch 35/188
Processed batch 36/188
Processed batch 37/188
Processed batch 38/188
Processed batch 39/188
Processed batch 40/188
Processed batch 41/188
Processed batch 42/188
Processed batch 43/188
Processed batch 44/1